<a href="https://colab.research.google.com/github/HofstraDoboli/project-2-project2/blob/main/Project2Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#for Project2 classification
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/Colab Notebooks/'
%ls

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks
 checkpoints/                   results/
'classification test.ipynb'     RL_Project.ipynb
'Copy of assignment6.ipynb'     Titanic-Dataset.csv
 LinRegProject2.ipynb           using_medrxivr.ipynb
 Project2Classification.ipynb   using_nfcorpus_duplicate.ipynb
 regressionData.csv             using_nfcorpus.ipynb


In [2]:
!pip install tensorboard
!pip install tensorflow-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.8/251.8 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 66.7 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.1 which is incompatible.
tensorflow 2.18.0 requires tensorboard<2.19,>=2.18, but you have tensorboard 2.19.0 which is incompatible.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
import tensorflow as tf
tf.autograph.set_verbosity(0)
from typing_extensions import runtime

Classification problem (NN learns to predict the class)

Titanic-Dataset:
The file should have exactly 2 columns:

PassengerId (sorted in any order)
Survived (contains your binary predictions: 1 for survived, 0 for deceased)

PassengerId,Survived:

892,0

893,1

894,0

Etc.


In [7]:
df.head()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,0,1,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,1,0,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,1,0,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,1,0,0,0,1
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,0,1,0,0,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        204 non-null    object 
 10  Sex_female   891 non-null    int64  
 11  Sex_male     891 non-null    int64  
 12  Embarked_C   891 non-null    int64  
 13  Embarked_Q   891 non-null    int64  
 14  Embarked_S   891 non-null    int64  
dtypes: float64(2), int64(10), object(3)
memory usage: 104.5+ KB


In [5]:
df = pd.read_csv("Titanic-Dataset.csv", thousands=',')

# One-hot encode categorical columns (creates Sex_male, Embarked_Q, Embarked_S, etc.)
df = pd.get_dummies(df, columns=['Sex', 'Embarked'], drop_first=False)

print(df.columns)

# Convert all columns with type 'uint8' or 'bool' to int
for col in df.columns:
    if df[col].dtype in ['uint8', 'bool']:
        df[col] = df[col].astype(int)


# Compute the correlation matrix
corr_matrix = df.select_dtypes(include=np.number).corr()
print("Correlation matrix:\n", corr_matrix)

# 2. EXTRACT OUTPUT COLUMN (Survived)
y_np = pd.get_dummies(df['Survived']).to_numpy()  # One-hot encode labels


# Get absolute correlation values with the output
output_corr = corr_matrix['Survived'].abs()

# Select features with correlation >= 0.2, excluding the output itself
selected_features = output_corr[output_corr >= 0.2].index.drop('Survived')
print("Selected features (corr >= 0.2):", selected_features)

# Create the absolute correlation matrix for selected features
corr_selected = df[selected_features].corr().abs()

# Select the upper triangle of the correlation matrix
upper = corr_selected.where(np.triu(np.ones(corr_selected.shape), k=1).astype(bool))

# Find columns with any correlation above 0.8
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]
print("Features to drop due to high intercorrelation:", to_drop)

# Final feature set
final_features = selected_features.drop(to_drop)
print("Final selected features:", final_features)

x_np = df[final_features].to_numpy()
print(x_np.shape)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Sex_female', 'Sex_male', 'Embarked_C',
       'Embarked_Q', 'Embarked_S'],
      dtype='object')
Correlation matrix:
              PassengerId  Survived    Pclass       Age     SibSp     Parch  \
PassengerId     1.000000 -0.005007 -0.035144  0.036847 -0.057527 -0.001652   
Survived       -0.005007  1.000000 -0.338481 -0.077221 -0.035322  0.081629   
Pclass         -0.035144 -0.338481  1.000000 -0.369226  0.083081  0.018443   
Age             0.036847 -0.077221 -0.369226  1.000000 -0.308247 -0.189119   
SibSp          -0.057527 -0.035322  0.083081 -0.308247  1.000000  0.414838   
Parch          -0.001652  0.081629  0.018443 -0.189119  0.414838  1.000000   
Fare            0.012658  0.257307 -0.549500  0.096067  0.159651  0.216225   
Sex_female     -0.042939  0.543351 -0.131900 -0.093254  0.114631  0.245489   
Sex_male        0.042939 -0.543351  0.131900  0.093254 -0.114631 -0.

In [8]:
### B. Normalize the data (subtract the mean and divide by the variance).
### Then split the data into training and validation (80% training/20% validation).


# 1. Split data into training and validation sets
# First split: 80% train, 20% temp (val+test)
x_np_train, x_np_temp, y_np_train, y_np_temp = train_test_split(
    x_np, y_np, test_size=0.2, shuffle=True, random_state=2
)

# Second split: 10% val, 10% test
x_np_val, x_np_test, y_np_val, y_np_test = train_test_split(
    x_np_temp, y_np_temp, test_size=0.5, shuffle=True, random_state=2
)

print("Train shape:", x_np_train.shape)
print("Val shape:", x_np_val.shape)
print("Test shape:", x_np_test.shape)

# 2. Normalize features (fit only on training)
x_scaler = StandardScaler()
x_np_train_scaled = x_scaler.fit_transform(x_np_train)
x_np_val_scaled   = x_scaler.transform(x_np_val)
x_np_test_scaled  = x_scaler.transform(x_np_test)

# 3. Normalize labels (fit only on training)
y_scaler = StandardScaler()
y_np_train_scaled = y_scaler.fit_transform(y_np_train)
y_np_val_scaled   = y_scaler.transform(y_np_val)
y_np_test_scaled  = y_scaler.transform(y_np_test)

# 4. Convert to PyTorch tensors
x_train_tensor = torch.from_numpy(x_np_train_scaled).float()
x_val_tensor   = torch.from_numpy(x_np_val_scaled).float()
x_test_tensor  = torch.from_numpy(x_np_test_scaled).float()

y_train_indices = torch.from_numpy(np.argmax(y_np_train, axis=1)).long()
y_val_indices   = torch.from_numpy(np.argmax(y_np_val, axis=1)).long()
y_test_indices  = torch.from_numpy(np.argmax(y_np_test, axis=1)).long()

train_data = [(x_train_tensor[i], y_train_indices[i]) for i in range(x_train_tensor.shape[0])]
val_data   = [(x_val_tensor[i],   y_val_indices[i])   for i in range(x_val_tensor.shape[0])]
test_data  = [(x_test_tensor[i],  y_test_indices[i])  for i in range(x_test_tensor.shape[0])]

trainloader = DataLoader(train_data, batch_size=32, shuffle=True)
valloader   = DataLoader(val_data, batch_size=8)
testloader  = DataLoader(test_data, batch_size=8)

Train shape: (712, 3)
Val shape: (89, 3)
Test shape: (90, 3)


In [9]:
# C. Train a NN for each problem. Compute the training loss and the validation loss as in the examples we did in class.
# Try different architectures: vary the number of hidden layers, number of hidden neurons, learning rate, optimization algorithm (SGD or Adam).
# Compare the models in terms of training and validation error. Comment on which models gave you better results and why.

In [10]:
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 15)
        self.layer2 = nn.Linear(15, 12) #2 hidden layers
        self.layer3 = nn.Linear(12, 2)  # 2 output classes (survived/died)
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x

In [25]:
model = Model(x_train_tensor.shape[1])
optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

def train_model(n_epochs, model, train_loader, val_loader, optimizer, loss_fn):
    for epoch in range(n_epochs+1):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for xb, yb in train_loader:
            optimizer.zero_grad()
            ym = model(xb)
            loss = loss_fn(ym, yb)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(ym.data, 1)
            total += yb.size(0)
            correct += (predicted == yb).sum().item()
        acc = 100 * correct / total
        train_loss = running_loss / len(train_loader)
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Train Loss: {train_loss:.4f}, Train Acc: {acc:.2f}%")
        # Validation
        if epoch % 10 == 0:
            model.eval()
            val_loss = 0.0
            val_correct = 0
            val_total = 0
            with torch.no_grad():
                for xb, yb in val_loader:
                    ym_val = model(xb)
                    loss_val = loss_fn(ym_val, yb)
                    val_loss += loss_val.item()
                    _, predicted = torch.max(ym_val.data, 1)
                    val_total += yb.size(0)
                    val_correct += (predicted == yb).sum().item()
            val_acc = 100 * val_correct / val_total
            val_loss = val_loss / len(val_loader)
            print(f"         Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    print("Training complete.")
    return train_loss, val_loss

train_loss, val_loss = train_model(
    n_epochs=100,
    model=model,
    train_loader=trainloader,
    val_loader=valloader,
    optimizer=optimizer,
    loss_fn=loss_fn
)

Epoch 0, Train Loss: 0.5616, Train Acc: 72.19%
         Val Loss: 0.4506, Val Acc: 78.65%
Epoch 10, Train Loss: 0.4285, Train Acc: 81.32%
         Val Loss: 0.4268, Val Acc: 79.78%
Epoch 20, Train Loss: 0.4539, Train Acc: 79.78%
         Val Loss: 0.4036, Val Acc: 80.90%
Epoch 30, Train Loss: 0.4385, Train Acc: 80.34%
         Val Loss: 0.4056, Val Acc: 82.02%
Epoch 40, Train Loss: 0.4313, Train Acc: 81.32%
         Val Loss: 0.4002, Val Acc: 79.78%
Epoch 50, Train Loss: 0.4339, Train Acc: 80.48%
         Val Loss: 0.4587, Val Acc: 75.28%
Epoch 60, Train Loss: 0.4253, Train Acc: 81.04%
         Val Loss: 0.3810, Val Acc: 82.02%
Epoch 70, Train Loss: 0.4212, Train Acc: 81.04%
         Val Loss: 0.4158, Val Acc: 79.78%
Epoch 80, Train Loss: 0.4274, Train Acc: 81.18%
         Val Loss: 0.4139, Val Acc: 82.02%
Epoch 90, Train Loss: 0.4128, Train Acc: 81.74%
         Val Loss: 0.4003, Val Acc: 82.02%
Epoch 100, Train Loss: 0.4187, Train Acc: 81.46%
         Val Loss: 0.3912, Val Acc: 82.02%

In [26]:
import torch as t

soft_max = t.nn.Softmax(dim=1)

# Training set predictions
y_m_train_pred = soft_max(model(x_train_tensor))
y_m_train_pred = t.argmax(y_m_train_pred, dim=1)

# Validation set predictions
y_m_val_pred = soft_max(model(x_val_tensor))
y_m_val_pred = t.argmax(y_m_val_pred, dim=1)

# Accuracy calculations
correct_pred_train = t.sum(y_m_train_pred == y_train_indices) / y_m_train_pred.shape[0]
correct_pred_val = t.sum(y_m_val_pred == y_val_indices) / y_m_val_pred.shape[0]

print(f"Final Training Loss: {train_loss:.4f}")
print(f"Final Validation Loss: {val_loss:.4f}")

print("Final Training accuracy = ", correct_pred_train.item())
print("Final Validation accuracy = ", correct_pred_val.item())

Final Training Loss: 0.4187
Final Validation Loss: 0.3912
Final Training accuracy =  0.8202247023582458
Final Validation accuracy =  0.8202247023582458


In [27]:
def evaluate_model(model, test_loader):
    model.eval()
    test_loss = 0.0
    test_correct = 0
    test_total = 0
    loss_fn = nn.CrossEntropyLoss()
    with torch.no_grad():
        for xb, yb in test_loader:
            ym = model(xb)
            loss = loss_fn(ym, yb)
            test_loss += loss.item()
            _, predicted = torch.max(ym.data, 1)
            test_total += yb.size(0)
            test_correct += (predicted == yb).sum().item()
    test_acc = 100 * test_correct / test_total
    test_loss = test_loss / len(test_loader)
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
    return test_loss, test_acc

# After training:
evaluate_model(model, testloader)


Test Loss: 0.5773, Test Acc: 72.22%


(0.577253465851148, 72.22222222222223)

testing:

In [13]:
class FlexibleModel(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim=2):
        """
        input_dim: int, number of input features
        hidden_layers: list of int, e.g. [15, 12] for two layers with 15 and 12 neurons
        output_dim: int, number of output classes (2 for Titanic)
        """
        super(FlexibleModel, self).__init__()
        layers = []
        prev_dim = input_dim
        for h in hidden_layers:
            layers.append(nn.Linear(prev_dim, h))
            layers.append(nn.ReLU())
            prev_dim = h
        layers.append(nn.Linear(prev_dim, output_dim))  # Output layer
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)


In [22]:
# 2 hidden layers with 15 and 12 neurons-my current setup
model1 = FlexibleModel(input_dim=x_train_tensor.shape[1], hidden_layers=[15, 12])

# 2 hidden layers with
model2 = FlexibleModel(input_dim=x_train_tensor.shape[1], hidden_layers=[20, 30, 8])

# 3 hidden layers with 32, 16, and 8 neurons
model3 = FlexibleModel(input_dim=x_train_tensor.shape[1], hidden_layers=[32, 16, 8])


In [23]:
optimizer = torch.optim.SGD(model1.parameters(), lr=0.05, momentum=0.9)
train_model(
    n_epochs=100,
    model=model1,
    train_loader=trainloader,
    val_loader=valloader,
    optimizer=optimizer,
    loss_fn=loss_fn
)

optimizer = torch.optim.SGD(model2.parameters(), lr=0.05, momentum=0.9)
train_model(
    n_epochs=100,
    model=model2,
    train_loader=trainloader,
    val_loader=valloader,
    optimizer=optimizer,
    loss_fn=loss_fn
)

optimizer = torch.optim.SGD(model3.parameters(), lr=0.05, momentum=0.9)
train_model(
    n_epochs=100,
    model=model3,
    train_loader=trainloader,
    val_loader=valloader,
    optimizer=optimizer,
    loss_fn=loss_fn
)


Epoch 0, Train Loss: 0.5857, Train Acc: 69.52%
         Val Loss: 0.4096, Val Acc: 80.90%
Epoch 10, Train Loss: 0.4399, Train Acc: 81.18%
         Val Loss: 0.3961, Val Acc: 80.90%
Epoch 20, Train Loss: 0.4257, Train Acc: 81.18%
         Val Loss: 0.4272, Val Acc: 82.02%
Epoch 30, Train Loss: 0.4367, Train Acc: 81.18%
         Val Loss: 0.4209, Val Acc: 79.78%
Epoch 40, Train Loss: 0.4425, Train Acc: 79.63%
         Val Loss: 0.4286, Val Acc: 82.02%
Epoch 50, Train Loss: 0.4240, Train Acc: 80.76%
         Val Loss: 0.4244, Val Acc: 75.28%
Epoch 60, Train Loss: 0.4262, Train Acc: 82.30%
         Val Loss: 0.4056, Val Acc: 79.78%
Epoch 70, Train Loss: 0.4376, Train Acc: 80.62%
         Val Loss: 0.3731, Val Acc: 80.90%
Epoch 80, Train Loss: 0.4313, Train Acc: 81.04%
         Val Loss: 0.3792, Val Acc: 82.02%
Epoch 90, Train Loss: 0.4174, Train Acc: 80.34%
         Val Loss: 0.3983, Val Acc: 82.02%
Epoch 100, Train Loss: 0.4246, Train Acc: 81.04%
         Val Loss: 0.4112, Val Acc: 79.78%

(0.43610338607560034, 0.35803106054080064)

In [24]:
model1.eval()
with torch.no_grad():
    y_m_train = model1(x_train_tensor)
    y_m_train = torch.argmax(y_m_train, dim=1)
    train_acc = (y_m_train == y_train_indices).float().mean().item()
    print("Final Training accuracy = ", train_acc)

    y_m_val = model1(x_val_tensor)
    y_m_val = torch.argmax(y_m_val, dim=1)
    val_acc = (y_m_val == y_val_indices).float().mean().item()
    print("Final Validation accuracy = ", val_acc)


model2.eval()
with torch.no_grad():
    y_m_train = model2(x_train_tensor)
    y_m_train = torch.argmax(y_m_train, dim=1)
    train_acc = (y_m_train == y_train_indices).float().mean().item()
    print("Final Training accuracy = ", train_acc)

    y_m_val = model2(x_val_tensor)
    y_m_val = torch.argmax(y_m_val, dim=1)
    val_acc = (y_m_val == y_val_indices).float().mean().item()
    print("Final Validation accuracy = ", val_acc)

model3.eval()
with torch.no_grad():
    y_m_train = model3(x_train_tensor)
    y_m_train = torch.argmax(y_m_train, dim=1)
    train_acc = (y_m_train == y_train_indices).float().mean().item()
    print("Final Training accuracy = ", train_acc)

    y_m_val = model3(x_val_tensor)
    y_m_val = torch.argmax(y_m_val, dim=1)
    val_acc = (y_m_val == y_val_indices).float().mean().item()
    print("Final Validation accuracy = ", val_acc)


Final Training accuracy =  0.8146067261695862
Final Validation accuracy =  0.7977527976036072
Final Training accuracy =  0.817415714263916
Final Validation accuracy =  0.8202247023582458
Final Training accuracy =  0.8160112500190735
Final Validation accuracy =  0.8202247023582458
